In [1]:
import sys

sys.path.append("../../")

In [2]:
import pandas as pd
import shared
import xarray as xr
from shared import open_dataset, save

## Load and aggregate AR6 FACTS projections

In [3]:
all_ds = []
global_ds = []
global_ssps = []


def open_and_convert(ds_path):
    out = open_dataset(ds_path)
    out["sea_level_change"] = (
        out.sea_level_change.pint.quantify().pint.to("meters").pint.dequantify()
    )
    return out


for kind in ["total", "verticallandmotion"]:
    ds = []
    this_ssps = []
    for conf in ["low", "medium"]:
        for ds_path in (shared.DIR_SLR_AR6_RAW / "regional").glob(f"{kind}_*_{conf}_*"):
            this_ssp = ds_path.name.split("_")[1]
            ssp_conf = f"{this_ssp}_{conf}"
            ds.append(open_and_convert(ds_path))
            this_ssps.append(ssp_conf)
            if kind == "total":
                global_ds.append(
                    open_and_convert(shared.DIR_SLR_AR6_RAW / "global" / ds_path.name)
                )
                global_ssps.append(ssp_conf)
    all_ds.append(
        xr.concat(ds, pd.Index(this_ssps, name="scenario"), data_vars="different")
    )

# for some reason the VLM dataset has an entry for 2005 that is all 0s, while other
# datasets just don't have 2005 b/c it is the assumed basline
assert (all_ds[1].sea_level_change.sel(years=2005) == 0).all()
all_ds[1] = all_ds[1].sel(years=slice(2006, None))

global_ds = (
    xr.concat(global_ds, pd.Index(global_ssps, name="scenario"), data_vars="different")
    .squeeze(drop=True)
    .drop_vars(["lon", "lat"])
    .sea_level_change
)

# handle floating point matching errors on the quantile dimension
global_ds["quantiles"] = all_ds[0].quantiles
all_ds[1]["quantiles"] = all_ds[0].quantiles

all_ds = xr.Dataset(
    {
        "lsl_msl05": all_ds[0].sea_level_change,
        "lsl_ncc_msl05": all_ds[1].sea_level_change,
        "gsl_msl05": global_ds,
        "lon": all_ds[1].lon,
        "lat": all_ds[0].lat,
    }
)

# drop locations with NaN values in the time period we're interested in
valid = (
    all_ds[["lsl_msl05", "lsl_ncc_msl05"]]
    .sel(years=slice(2100))
    .notnull()
    .all(["scenario", "quantiles", "years"])
    .to_array("tmp")
    .all("tmp")
)
all_ds = all_ds.sel(locations=valid)

all_ds = all_ds.rename(
    {"years": "year", "quantiles": "quantile", "locations": "site_id"}
)

# we generally allow +180 but not -180
all_ds["lon"] = all_ds.lon.where(all_ds.lon != -180, 180)

# drop locations where we have missing values
valid = all_ds.sel(year=slice(2100)).notnull().all(["scenario", "quantile", "year"])

## Save

In [ ]:
for v in valid.variables:
    valid[v].encoding.clear()
save(valid.chunk({"site_id": 100}), shared.PATH_SLR_AR6, mode="w")